In [27]:
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import numpy as np
plt.style.use(['seaborn-colorblind','paper'])
plt.style.use('classic')
import Cosmology as Cosmo
from scipy.integrate import simps
from scipy.interpolate import interp1d

In [28]:
def dNdz_LSST(bin_num,dn_filename = 'dndz_LSST_i27_SN5_3y'):
    if bin_num is "all":
        zbin, nbin = pickle.load(open(dn_filename+'tot_extrapolated.pkl','r'))
        norm                = simps(nbin,zbin)
        mbin                = 'all'
    else:
        bins,big_grid,res   = pickle.load(open(dn_filename+'_extrapolated.pkl','r'))
        mbin                = bins[bin_num]
        zbin                = big_grid
        nbin                = res[bin_num]
        norm                = simps(nbin,zbin)
        dndz                = interp1d(zbin, nbin/norm, kind='linear',bounds_error=False,fill_value=0.)
    print 'using z-bin', mbin
    return norm

In [37]:
"""------------settings----------------"""
bin_num     = 'all'

fsky        = 0.5



params      = Cosmo.Planck2015
tag         = params[0]['name']
nl          = True

N0_path     ='/home/nessa/Documents/Projects/LensingBispectrum/CMB-nonlinear/outputs/N0files/'
spec_path   ='/home/nessa/Documents/Projects/LensingBispectrum/CMB-nonlinear/outputs/power_spectra/'
ellpath     ='/home/nessa/Documents/Projects/LensingBispectrum/CMB-nonlinear/outputs/ells/'
ellfile     ='ell_ang_full_Lmin1_Lmax3000_lmin1_lmax8000_lenL120_lenl140_lenang120_1e-04.pkl'
path        ='/home/nessa/Documents/Projects/LensingBispectrum/CMB-nonlinear/outputs/noise_curves/'

#bispectrum code
config_kk = 'kk'
config_kg = 'kg'
config_gg = 'gg'
Lmin = 1
Lmax = 3000

thetaFWHMarcmin = 3. #beam FWHM
noiseUkArcmin   = 0.5 #eval(sys.argv[1]) #Noise level in uKarcmin
l_max_T         = 3000
l_max_P         = 5000
l_min           = 50
L_max           = 6000 #for l integration
L_min           = 1
TCMB            = 2.7255e6
div             = True #divide EB by factor of 2.5
channel         = 'mv'
exp             = 'Nk'

if l_max_T!=l_max_P:
    lmax='mixedlmax_%d_%d'%(l_max_T,l_max_P)
else:
    lmax=str(l_max_T)

if nl:
  nl_='_nl'
else:
  nl_=''

if div:
    print 'Dividing EB by factor 2.5!'
    no_div='div25'
else:
    no_div='nodiv'

Dividing EB by factor 2.5!


In [38]:
print N0_path+'%s_N0_%s_%d_%d%d_%s%s.pkl'%(tag,lmax,l_min,10*noiseUkArcmin,10*thetaFWHMarcmin,no_div,nl_)
filename=N0_path+'%s_N0_%s_%d_%d%d_%s%s.pkl'%(tag,lmax,l_min,10*noiseUkArcmin,10*thetaFWHMarcmin,no_div,nl_)
Ls,AL = pickle.load(open(filename,'r'))

/home/nessa/Documents/Projects/LensingBispectrum/CMB-nonlinear/outputs/N0files/Planck2015_N0_mixedlmax_3000_5000_50_530_div25_nl.pkl


In [39]:
#percent accuracy, should be sufficient for this
config_kk+="_full_"+tag
tag = "_"+"LSSTbin"+str(bin_num)+'_full_'+tag
config_kg+=tag
config_gg+=tag
ll, l, theta = pickle.load(open(ellpath+ellfile,'r'))
clgg = np.load(spec_path+"CL_%s_Lmin%d-Lmax%d_%s.npy"%(config_gg,Lmin,Lmax-1,'halofit'))
clkg = np.load(spec_path+"CL_%s_Lmin%d-Lmax%d_%s.npy"%(config_kg,Lmin,Lmax-1,'halofit'))
clkk = np.load(spec_path+"CL_%s_Lmin%d-Lmax%d_%s.npy"%(config_kk,Lmin,Lmax-1,'halofit'))


In [40]:
L = np.arange(max(min(Ls),min(L),min(ll)),min(max(Ls),max(L),max(ll)))
clkk = np.interp(L,ll,clkk)
clgg = np.interp(L,ll,clgg)
clkg = np.interp(L,ll,clkg)
n0  = np.interp(L,Ls,1./4.*(Ls*(Ls+1))**2*AL[channel])
norm=dNdz_LSST(bin_num)
n_bar = norm*(180*60/np.pi)**2
print (norm,n_bar)
noise_kk      = clkk+n0
noise_gg      = clgg+1./n_bar

pickle.dump([L,noise_gg,noise_kk],open(path+'Gaussian_variances_4SN_%s_%s_%s%s_%s.pkl'%(channel,exp,lmax,tag,'dndz_LSST_i27_SN5_3y'),'w'))
print(path+'Gaussian_variances_4SN_%s_%s_%s%s_%s.pkl'%(channel,exp,lmax,tag,'dndz_LSST_i27_SN5_3y'))
print(min(L),max(L))

using z-bin all
(65.672302580174119, 776122026.94831777)
/home/nessa/Documents/Projects/LensingBispectrum/CMB-nonlinear/outputs/noise_curves/Gaussian_variances_4SN_mv_Nk_mixedlmax_3000_5000_LSSTbinall_full_Planck2015_dndz_LSST_i27_SN5_3y.pkl
(1.0, 2996.0)
